## Final report outline, and quick notes

- In Monte carlo simulation you are really looking to reduce the standard error of your approximation.
  - This is why the Predicted price is meaningless, however the **standard error** is **very important**
- A main Drawback of the balck-scholes model is that it assumes a **constant volatility**
  - Discuss the assumptions of the BS model and how they help or hurt it. 
    - Stock follows a lognormal distribution
    - stock prices can not be negative 
    - no dividends are paid 
    - frictionless market 
    - constant volatility
- Get data from a few different Contracts, to compare the results.  
  - Lets try Short term, Mid term, and Long term contracts. Mind you this gives you a chance to excercise the option each day. 
    - Short term: 60 days 
    - Medium term: 180 days 
    - Long Term: 360 days

**Main Idea:** 
1. Why are we pricing options? 
   1. We want to find the fair value of the option today
   2. This will help us determine if an option is overpriced or not. 
2. Compare simulated, numerical, and predicted results to the observed prices.
   1. Observed prices are set by the market maker, so we can also evaluate how much the market maker is charging for the option.
3. Compare the standard error of the predicted prices to the standard error of the observed prices.
 
----

Outline for the final presentation (from the paper referenced below)
1. Present a simple numerical example of the simulation apprach 
2. Describe the undderlying theoretical framework 
3. Provide specific erxamples of the application of this approach 
4. discuss number of numerical and implementation issues
5. summarize and give concluding remarks.


**Notes From The paper:**
- On the exercise date, the optimal exercise strategy for an American Option is to do so if the option is in the money. Prior to the exercise date, the optimal strategy is to compare the immediate exercise value with the expected cashflows from continuing to the option until the exercise date. If the immediate value is greater than the expected value at expiration, we exercise the option. Else we continue to the expiration date.
- The holder of an American option optimally compares the payoff from the immediate exercise with the expected payoff from continutation, and then chooses to exercise if the immediate payoff is higher. 
  - The key to optimaly exercise an american option is identifying the conditional expected value of the option at the exercise date.
    - The optimal exercise stategy is fundamentally determined by the conditional expectation of the payoff from continuing to keep the option alive. 
- A key insight from the Longstaff-Schwartz paper is that this conditional expectation can be estimated from the cross-sectional information in the simulation by using the least squares method. 
    -  More specifically Longstaff and Schwartz show that:
       -  The ex post realized payoffs from continuation on functions of the values of the state variables on functions of the values of the state variables. 
       -  The fitted value from this regression provides a direct estimate of the conditional expectation function
       -  By estimating the conditional expectation function for each exercise date, we obtain a complete specification of the optimal exercise strategy for the option, along each of the simulated paths. 
-  Another key insight is that the paper only only includes the paths for which the **option is in the money**. 

**Source:** *"Valuing American Options by Simulation: A Simple Least-Squares Approach, Francis A. Longstaff, and Eduardo S. Schwartz UCLA "*

----

In [1]:
import re
import datetime as dt 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from IPython.display import display
from tqdm import tqdm
import sqlite3 as sql 
from scipy.optimize import minimize
import scipy.stats as st 
%matplotlib inline


How to take in account Early Excersise?

1. Simulate the underlying asset price paths using a stochastic process such as geometric Brownian motion.
2. At each time step, determine the optimal exercise decision for each path by comparing the immideate value of the option to the value if you were to  con to hold the option.
3. Use regression techniques to estimate the continuation value of the option as a function of the simulated asset prices, exercise decisions, and other relevant variables.
4. Backward-induct to the beginning of the option's life to obtain the option's fair value.
5. Repeat the simulation and valuation process many times to obtain a more accurate estimate of the option's fair value.

---- 
# American Test 

The input parameters for this function are:
- `maturity`: the time to maturity of the option (in years)
- `strike_price`: the strike price of the option
- `spot_price`: the current spot price of the underlying asset
- `volatility`: the volatility of the underlying asset's returns
- `risk_free_rate`: the risk-free interest rate
- `num_sims`: the number of Monte Carlo simulations to run
- `num_time_steps`: the number of time steps to use in the simulation
- `option_type`: the type of option, either "call" or "put"

The function returns the estimated price of the option.

In [17]:
# Longstaff-Schwartz Monte Carlo American Option Pricing
import numpy as np
from numpy.polynomial import Polynomial

# X: matrix of simulated paths
# t: vector of time steps
# df: discount factor function
# fit: function to fit continuation value
# exercise_payoff: function to compute exercise payoff
# itm_select: function to select in-the-money paths

def longstaff_schwartz_iter(X, t, df, fit, exercise_payoff, itm_select=None):
    # given no prior exercise we just receive the final payoff
    cashflow = exercise_payoff(X[-1, :])
    # iterating backwards in time
    for i in reversed(range(1, X.shape[0] - 1)):
        # discount cashflows from next period
        cashflow = cashflow * df(t[i], t[i + 1])
        # stock price at time t[i]
        x = X[i, :]
        # exercise value for time t[i]
        exercise = exercise_payoff(x)
        # boolean index of all in-the-money paths
        # (paths considered for exercise)
        itm = itm_select(exercise, x) if itm_select else np.full(x.shape, True)
        # fit curve
        fitted = fit(x[itm], cashflow[itm])
        # approximate continuation value
        continuation = fitted(x)
        # boolean index where exercise is beneficial
        ex_idx = itm & (exercise > continuation)
        # update cashflows with early exercises
        cashflow[ex_idx] = exercise[ex_idx]

        yield cashflow, x, fitted, continuation, exercise, ex_idx
        
def longstaff_schwartz(X, t, df, fit, exercise_payoff, itm_select=None):
    # iterate over time steps
    for cashflow, *_ in longstaff_schwartz_iter(
        X, t, df, fit, exercise_payoff, itm_select
    ):
        pass
    # return discounted average of cashflows
    return cashflow.mean(axis=0) * df(t[0], t[1])

def ls_american_option_quadratic_iter(X, t, r, strike):
    # given no prior exercise we just receive the payoff of a European option
    cashflow = np.maximum(strike - X[-1, :], 0.0)
    # iterating backwards in time
    for i in reversed(range(1, X.shape[0] - 1)):
        # discount factor between t[i] and t[i+1]
        df = np.exp(-r * (t[i + 1] - t[i]))
        # discount cashflows from next period
        cashflow = cashflow * df
        x = X[i, :]
        # exercise value for time t[i]
        exercise = np.maximum(strike - x, 0.0)
        # boolean index of all in-the-money paths
        itm = exercise > 0
        # fit polynomial of degree 2
        fitted = Polynomial.fit(x[itm], cashflow[itm], 2)
        # approximate continuation value
        continuation = fitted(x)
        # boolean index where exercise is beneficial
        ex_idx = itm & (exercise > continuation)
        # update cashflows with early exercises
        cashflow[ex_idx] = exercise[ex_idx]

        yield cashflow, x, fitted, continuation, exercise, ex_idx

def longstaff_schwartz_american_option_quadratic(X, t, r, strike):
    for cashflow, *_ in ls_american_option_quadratic_iter(X, t, r, strike):
        pass
    # return discounted average of cashflows
    return cashflow.mean(axis=0) * np.exp(-r * (t[1] - t[0]))

In [18]:
# Monte Carlo Simulation of the stock price paths to find Fair Value of the option
def mc_sim(S0, r, sigma, K, T, N, days, otype):
    
    Z = np.random.normal(size=N) # Standard Normal of size N, for error term
    ST = S0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*Z) # lognormal distribution of the price
    
    # Monte Carlo simulation of the stock price paths
    dt = T/days  
    S = np.zeros((days+1, N))  
    S[0] = S0  
    for t in range(1, days+1):
        Z = np.random.normal(size=N)
        S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z) # Geometric Brownian Motion

    # calculate the payoff at maturity
    if otype.lower() == 'call':
        hT = np.maximum(S[-1]-K, 0)
    elif otype.lower() == 'put':
        hT = np.maximum(K-S[-1], 0)

    # calculate the Monte Carlo estimator
    C0 = np.exp(-r*T)*np.sum(hT)/N
    return C0


In [19]:

N = 1000 # 2 million sims
S0 = 100
K = 105
r = 0.03454
sigma = .12
days = 100
otype = 'call' # option type
Observed = 5.00

# Step 1: Obtain Simulated stock price data, via geometric brownian motion
T = days / 252 
dt = T/days  
S = np.zeros((days+1, N))  
S[0] = S0  
for t in range(1, days+1):
    Z = np.random.normal(size=N)
    S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z) # Geometric Brownian Motion

# Run LSMC, to obtain fair option price at time 0 
t = np.linspace(0, dt, days+1)
lsm_fair_price = longstaff_schwartz_american_option_quadratic(S, t, r, K)
optimal_excercise_points = np.where(Observed > lsm_fair_price)

# Monte Carlo Simulation at expiration 
mc_fair_price = mc_sim(S0, r, sigma, K, T, N, days, otype)

# Black Scholes Comparison 
from scipy.stats import norm
d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
d2 = (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
bs_call =  (S0*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
if otype.lower() == 'put':
    bs_call = bs_call - S0 + K*np.exp(-r*T)


print(f'''
Option Type: {otype.capitalize()}
Stock Price Today: ${S0:.2f}
Strike Price: ${K:.2f}
Days to Expiration: {days}
Volatility: {sigma:.4f}

LSM Fair Value Today: \x1b[31m{lsm_fair_price:.4f}\x1b[0m
Monte Carlo Fair Value Today: \033[94m{mc_fair_price:.4f}\033[0m
Black Scholes Fair Value Today: \033[92m{bs_call:.4f}\033[0m
Observed Price Today: {Observed:.4f}
Simulated Possible Excercise points after today: {optimal_excercise_points[0].shape[0]}

''')


Option Type: Call
Stock Price Today: $100.00
Strike Price: $105.00
Days to Expiration: 100
Volatility: 0.1200

LSM Fair Value Today: 5.8739
Monte Carlo Fair Value Today: 1.6167
Black Scholes Fair Value Today: 1.6080
Observed Price Today: 5.0000
Simulated Possible Excercise points after today: 0




In [20]:
N = 100 # number of simulations
S0 = 100 # stock price today
K = 105 # strike price
r = 0.03454 # risk free rate 
sigma = .12 # volatility
T = days / 252 # time to expiration
otype = 'call' # option type
days = 50 # days to expiration
Observed = 5.00 # observed price today

# Step 1: Obtain Simulated stock price data, via geometric brownian motion
dt = T/days  
S = np.zeros((days+1, N))  
S[0] = S0  
for t in range(1, days+1):
    Z = np.random.normal(size=N)
    S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z) # Geometric Brownian Motion

# Run LSMC, to obtain fair option price at time 0 
t = np.linspace(0, T, days+1)
lsm_fair_price = longstaff_schwartz_american_option_quadratic(S, t, r, K)
optimal_excercise_points = np.where(Observed > lsm_fair_price)

# Monte Carlo Simulation at expiration 
mc_fair_price = mc_sim(S0, r, sigma, K, T, N, days, otype)


# Black Scholes Comparison 
# calculate the analytical European call option price
from scipy.stats import norm
d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
d2 = (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
bs_call =  (S0*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
if otype.lower() == 'put':
    bs_call = bs_call - S0 + K*np.exp(-r*T)


print(f'''
Option Type: {otype}
Volatility: {sigma:.4f}

Stock Price Today: ${S0:.2f}
Strike Price: ${K:.2f}

LSM Fair Value Today: \x1b[31m{lsm_fair_price:.4f}\x1b[0m
Black Scholes Fair Value Today: \033[92m{bs_call:.4f}\033[0m
Monte Carlo Fair Value Today: \033[94m{mc_fair_price:.4f}\033[0m
Observed Price Today: {Observed:.4f}
Simulated Possible Excercise points after today: {optimal_excercise_points[0].shape[0]}

''')


Option Type: call
Volatility: 0.1200

Stock Price Today: $100.00
Strike Price: $105.00

LSM Fair Value Today: 5.6991
Black Scholes Fair Value Today: 1.6080
Monte Carlo Fair Value Today: 1.6500
Observed Price Today: 5.0000
Simulated Possible Excercise points after today: 0




In [21]:
N = 100 # number of simulations
S0 = 100 # stock price today
K = 105 # strike price
r = 0.03454 # risk free rate 
sigma = .12 # volatility
T = days / 252 # time to expiration
otype = 'call' # option type
days = 50 # days to expiration
Observed = 5.00 # observed price today

# Step 1: Obtain Simulated stock price data, via geometric brownian motion
N = 100
dt = T/days  
S = np.zeros((days+1, N))  
S[0] = S0  
for t in range(1, days+1):
    Z = np.random.normal(size=N)
    S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z) # Geometric Brownian Motion

# Run LSMC, to obtain fair option price at time 0 
t = np.linspace(0, T, days+1)
lsm_fair_price = longstaff_schwartz_american_option_quadratic(S, t, r, K)
optimal_excercise_points = np.where(Observed > lsm_fair_price)

# Monte Carlo Simulation at expiration 
mc_fair_price = mc_sim(S0, r, sigma, K, T, N, days, otype)

# Black Scholes Comparison 
from scipy.stats import norm
d1 = (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
d2 = (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
bs_call =  (S0*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2))
if otype.lower() == 'put':
    bs_call = bs_call - S0 + K*np.exp(-r*T)


print(f'''
Option Type: {otype}
Volatility: {sigma:.4f}

Stock Price Today: ${S0:.2f}
Strike Price: ${K:.2f}

LSM Fair Value Today: \x1b[31m{lsm_fair_price:.4f}\x1b[0m
Black Scholes Fair Value Today: \033[92m{bs_call:.4f}\033[0m
Monte Carlo Fair Value Today: \033[94m{mc_fair_price:.4f}\033[0m
Observed Price Today: {Observed:.4f}
Simulated Possible Excercise points after today: {optimal_excercise_points[0].shape[0]}
''')


Option Type: call
Volatility: 0.1200

Stock Price Today: $100.00
Strike Price: $105.00

LSM Fair Value Today: 5.5916
Black Scholes Fair Value Today: 0.6740
Monte Carlo Fair Value Today: 0.5703
Observed Price Today: 5.0000
Simulated Possible Excercise points after today: 0

